In [1]:
import torch
from torchvision import models
state_dict = torch.load('resnet34.pth')
model = models.resnet34(pretrained=False)
model.load_state_dict(state_dict)
model.eval()

c:\Users\VIKTOR\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\VIKTOR\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

# Определение стандартных трансформаций
normalize = transforms.Normalize(mean=[0.485,  0.456,  0.406], std=[0.229,  0.224,  0.225])
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

# Путь к директории с изображениями
data_dir = './drones'
dataset = ImageFolder(data_dir, transform=transform)


In [4]:
from torch.utils.data import DataLoader

# Параметры DataLoader
batch_size =  64
shuffle = True

# Создание DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


In [ ]:
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 1000 +  1) 
# Заморозка нижних слоев модели
for param in model.parameters():
    param.requires_grad = False

# Размораживание верхних слоев для обучения
for param in model.fc.parameters():
    param.requires_grad = True

# Определение функции потерь и оптимизатора
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Обучение модели*
for epoch in range(num_epochs):
    running_loss =  0.0
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    print('Epoch {}/{}: Loss: {:.4f}'.format(epoch, num_epochs -  1, epoch_loss))
